In [1]:
from bs4 import BeautifulSoup
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import time
import datetime
import pandas as pd

In [ ]:
!which chromedriver

In [13]:
executable_path = {'executable_path': ChromeDriverManager().install()}
# browser = Browser('chrome', **executable_path)
browser = Browser('chrome', **executable_path, headless=False)
# browser = Browser('chrome')

[WDM] - Current google-chrome version is 84.0.4147
[WDM] - Get LATEST driver version for 84.0.4147


[WDM] - Driver [/Users/abby/.wdm/drivers/chromedriver/mac64/84.0.4147.30/chromedriver] found in cache


In [3]:
base_url = 'https://www.walmart.com'
search_url = '/search/?query=room%20air%20purifier'
browser.visit(base_url + search_url)

In [4]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [5]:
products = soup.find_all('div', class_='search-result-gridview-item')

In [7]:
# Create empty lists for scraped data to be stored in.
productURL_list = []
productImage_list = []
productTitle_list = []
starReview_list = []
reviewAmount_list = []
currentPrice_list = []
reviewURL_list = []


today = datetime.datetime.now()

for product in products:

    # Get the link to the product page.
    link = product.find('a')
    href = link['href']
    product_url = base_url + href
    productURL_list.append(product_url)

    # Get the link to the product image.
    img = product.find('img')['src']
    productImage_list.append(img)

    # Get the product title.
    product_title = product.find('img')['alt']
    productTitle_list.append(product_title)

    # Try to get the number of reviews, there may not be any.
    try:
        review_amount = product.find('span', class_='seo-review-count visuallyhidden').text
        reviewAmount_list.append(review_amount)
        
    except:
        review_amount = 0
        reviewAmount_list.append(review_amount)

    # In the case where there are at least one review...
    if int(review_amount) > 0:
        # Get the average number of stars
        stars_review = product.find('span', class_='visuallyhidden seo-avg-rating').text   
        starReview_list.append(stars_review)
        
        # Get the URL to the reviews section for that product.
        review_url = product.find('div', class_='stars').find('a')['href']
        reviewURL_list.append(base_url+review_url)

    # Otherwise, use defalt 0 or NaN values for these entries.
    else:
        stars_review = 0
        starReview_list.append(stars_review)
        
        review_url = "NaN"
        reviewURL_list.append(review_url)
        
    # Get the price of the product.
    price = product.find('span', class_='price-main')
    current_price = price.find('span', class_='visuallyhidden').text
    currentPrice_list.append(current_price)
    
# Close the browser window
# browser.quit()
    
# Create a dictionary with the lists of the scrapped data.
data = {
    "Title": productTitle_list,
    "URL": productURL_list,
    "Image": productImage_list,
    "AverageStars": starReview_list,
    "NumberofReviews": reviewAmount_list,
    "ReviewsURL": reviewURL_list,
    "Price": currentPrice_list
       }

# Create a Pandas DataFrame with that dictionary
product_df = pd.DataFrame.from_dict(data)
product_df.head()

,Title,URL,Image,AverageStars,NumberofReviews,ReviewsURL,Price
0,Holmes Personal Space 2-Speed Air Purifier wit...,https://www.walmart.com/ip/Holmes-Personal-Spa...,https://i5.walmartimages.com/asr/ca7e3b65-d3c8...,4.3,517,https://www.walmart.com/ip/Holmes-Personal-Spa...,$14.98
1,Holmes Desktop HEPA Air Purifier with Visipure...,https://www.walmart.com/ip/Holmes-Desktop-HEPA...,https://i5.walmartimages.com/asr/22b30c76-a2a4...,4.5,65,https://www.walmart.com/ip/Holmes-Desktop-HEPA...,$32.88
2,High-Efficiency HEPA-Type Desktop Air Purifier...,https://www.walmart.com/ip/High-Efficiency-HEP...,https://i5.walmartimages.com/asr/5ba9740e-a3f1...,4.4,136,https://www.walmart.com/ip/High-Efficiency-HEP...,$49.69
3,Winix 5500-2 Air Cleaner with PlasmaWave Techn...,https://www.walmart.com/ip/Winix-5500-2-Air-Cl...,https://i5.walmartimages.com/asr/7e1f8414-d0c2...,4.7,89,https://www.walmart.com/ip/Winix-5500-2-Air-Cl...,$159.99
4,Filtrete F2 Bacteria & Virus True HEPA Room Ai...,https://www.walmart.com/ip/Filtrete-F2-Bacteri...,https://i5.walmartimages.com/asr/dea94bac-be28...,4.8,55,https://www.walmart.com/ip/Filtrete-F2-Bacteri...,$17.97


In [11]:
product_df = product_df.astype({
    "NumberofReviews": 'int',
    "AverageStars": 'float'
})

product_df.dtypes

Title               object
URL                 object
Image               object
AverageStars       float64
NumberofReviews      int64
ReviewsURL          object
Price               object
dtype: object

In [41]:
# Get reviews 

review_df = pd.DataFrame()

# for i in range(len(product_df["ReviewsURL"])):
for i in range(4,5):
    if product_df['NumberofReviews'][i] > 0:
        
        productTitleList = []
        reviewURLList = []
        reviewTitleList = []
        reviewRatingList = []
        reviewCommentList = []
        
        if len(reviewRatingList) < product_df['NumberofReviews'][i]:
    
            try:
                browser.visit(product_df["ReviewsURL"][i])
                html = browser.html
                soup = BeautifulSoup(html, 'html.parser')
                allReviewsURL = soup.find('a', class_="button ReviewBtn-container ReviewsHeader-seeAll button--primary")['href']
                browser.visit(base_url+allReviewsURL)
                soup = BeautifulSoup(html, 'html.parser')
                reviews = soup.find_all('div', class_='ReviewList-content')

                # need to add a way to make sure I am capturing all of the reviews
                # can check this by looking at NumberofReview for this ReviewsRUL
                # to scroll down a little use the below command
                # browser.execute_script("window.scrollTo(200, document.body.scrollHeight);")
                # check the average rating by using the values found here



                for review in reviews:

                    productTitleList.append(product_df['Title'][0])
                    reviewURLList.append(product_df['ReviewsURL'][0])

                    # need to fix this, shouldn't need both try and if
                    try:
                        if (review.find('h3', class_='review-title').text):
                            title = review.find('h3', class_='review-title').text
                        elif (review.find('h3', class_='review-title').text) == 'None':
                            title = 'No Title'
                        else:
                            title = 'No Text'
                            # if I can get this part to work ..
                            # this is where I can add the scroll down
                            # then try to grab the title again
                            pass
                        reviewTitleList.append(title)
                        print(title)

                    except:
                        title = 'None'
                        reviewTitleList.append(title)
                        print(title)
#                         pass

                    # adding try only because the browser needs to scroll
                    # once this is added, there wont need to be a try
                    # each review has to have a star value
                    try:
                        reviewRating = review.find('span', class_='seo-avg-rating').text
                        reviewRatingList.append(reviewRating)
                        print(reviewRating)
                    except:
                        pass

                    # adding try and except for the review body for same reason as stars
                    try:
                        reviewComment = review.find('p').text
                        reviewCommentList.append(reviewComment)
#                         print(reviewComment)
                    except:
                        pass

                data = {
                    'ProductTitle': productTitleList,
                    'ReviewURL': reviewURLList,
                    'ReviewTitle': reviewTitleList,
                    'ReviewStarRating': reviewRatingList,
                    'ReviewComment': reviewCommentList
                }
                print('stored data')
                reviewsDF = pd.DataFrame.from_dict(data)
                reviewsDF.head()
                print('into dataframe')
                review_df = review_df.append(reviewsDF)

            except:
                pass
#                 browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

None
5.0
A Perfect Example of an Air Filter at its Best!!
5.0
Breathing much easier with the F2 HEPA filter
5.0
Allergens be gone!
5.0
Great Product
5.0
Fabulous Filter
5.0
Making a great product even better!
5.0
None
5.0
Worth it.  Where does all this stuff come from?
4.0
Work as expected. Better Instructions needed.
3.0
stored data
into dataframe


In [42]:
# len(review_df)
review_df.head()
# productReview_df.head()

,ProductTitle,ReviewURL,ReviewTitle,ReviewStarRating,ReviewComment
0,Holmes Personal Space 2-Speed Air Purifier wit...,https://www.walmart.com/ip/Holmes-Personal-Spa...,None,5.0,So far so good. We all see if they last 6 months.
1,Holmes Personal Space 2-Speed Air Purifier wit...,https://www.walmart.com/ip/Holmes-Personal-Spa...,A Perfect Example of an Air Filter at its Best!!,5.0,This air filter from Filtrete is most impressi...
2,Holmes Personal Space 2-Speed Air Purifier wit...,https://www.walmart.com/ip/Holmes-Personal-Spa...,Breathing much easier with the F2 HEPA filter,5.0,We received this as a free sample from 3M for ...
3,Holmes Personal Space 2-Speed Air Purifier wit...,https://www.walmart.com/ip/Holmes-Personal-Spa...,Allergens be gone!,5.0,I received this filter free from 3M to evaluat...
4,Holmes Personal Space 2-Speed Air Purifier wit...,https://www.walmart.com/ip/Holmes-Personal-Spa...,Great Product,5.0,This filter works exactly as it should. It is ...


In [ ]:
reviews = soup.find('div', class_='review-highlight')
positive = reviews.find('div', class_='font-bold highlight-title').text
stars = reviews.find('span', class_='seo-avg-rating').text
# starts = reviews.find('span', class_='seo-average-rating')
body = reviews.find('div', class_='collapsable-content-container').text
print(positive)
print(stars)
print(body)

In [ ]:
reviews = soup.find_all('div', class_='ReviewList-content')

In [ ]:
# need to add a way to make sure I am capturing all of the reviews
# can check this by looking at NumberofReview for this ReviewsRUL
# to scroll down a little use the below command
# browser.execute_script("window.scrollTo(200, document.body.scrollHeight);")
# check the average rating by using the values found here

productTitleList = []
reviewURLList = []
reviewTitleList = []
reviewRatingList = []
reviewCommentList = []

for review in reviews:
    
    productTitleList.append(product_df['Title'][0])
    reviewURLList.append(product_df['ReviewsURL'][0])
    
    # need to fix this, shouldn't need both try and if
    try:
        if (review.find('h3', class_='review-title').text):
            title = review.find('h3', class_='review-title').text
            reviewTitleList.append(title)
#             print(title)
        elif (review.find('h3', class_='review-title').text) == 'None':
            title = 'None'
            reviewTitleList.append(title)
#             print(title)
        else:
            # if I can get this part to work ..
            # this is where I can add the scroll down
            # then try to grab the title again
            pass
        
    except:
        pass
    
    # adding try only because the browser needs to scroll
    # once this is added, there wont need to be a try
    # each review has to have a star value
    try:
        reviewRating = review.find('span', class_='seo-avg-rating').text
        reviewRatingList.append(reviewRating)
#         print(reviewRating)
    except:
        pass
    
    # adding try and except for the review body for same reason as stars
    try:
        reviewComment = review.find('p').text
        reviewCommentList.append(reviewComment)
#         print(reviewComment)
    except:
        pass
    
ddata = {
    'ProductTitle': productTitleList,
    'ReviewURL': reviewURLList,
    'ReviewTitle': reviewTitleList,
    'ReviewStarRating': reviewRatingList,
    'ReviewComment': reviewCommentList
}

review_df = pd.DataFrame(data)
review_df.head()

In [ ]:
browser.quit()

In [ ]:
soup.execute_script("window.scrollTo(0, document.body.scrollHeight);")